#1. Tunning de parametros para rede LSTM
##2. especificações do tunning

Camadas ocultas|Neuronios|Taxa de aprendizagem|Função de aprendizagem| Epocas
---------------|---------|--------------------|---------|---------|
        1      |  10-x+5-100   | 0.001 - 0.01       |Relu| 10-x+10- 100      
  
        
##3. importar as bibliotecas

###3.1 instalar a biblioteca do keras prórpia para o tunning



In [ ]:
pip install -q -U keras-tuner

###3.2 importar todas as bibliotecas que serão usadas no processo

In [ ]:

import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner as kt

##4. Importar os dataset e deixar o dataset na estrutura de entrada

In [ ]:
df_teste = pd.read_parquet('/content/drive/MyDrive/df_teste_norm')
df_treino = pd.read_parquet('/content/drive/MyDrive/df_treino_norm')
df_treino = df_treino.dropna()
df_teste = df_teste.dropna()
df_treino2 = df_treino[['TEMP_NORM','BATIMETRIA_NORM']]
df_teste2 = df_teste[['TEMP_NORM','BATIMETRIA_NORM']]
lista = list()
for x,i in df_treino2.values:
  lista.append([x,i])
sal_treino = np.array(df_treino.SALINIDADE)
pr_treino = np.array(lista)
lista = list()
for x,i in df_teste2.values:
  lista.append([x,i])
sal_teste = np.array(df_teste.SALINIDADE)
pr_teste = np.array(lista)
pr_teste = pr_teste.reshape(pr_teste.shape[0],pr_teste.shape[1],1)
pr_treino = pr_treino.reshape(pr_treino.shape[0],pr_treino.shape[1],1)

## 5. Fazer o tunning

In [ ]:
def KerasTunnerLSTM1(hp):
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  hiper_param = hp.Int('units', min_value=10, max_value=100, step=5)
  lstm = tf.keras.layers.LSTM(units=hiper_param,activation='relu')(input)
  output = tf.keras.layers.Dense(1,name='salinidade')(lstm)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  taxa_apren =  hp.Choice('learning_rate', values=[0.001,0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01])
  adan = tf.keras.optimizers.Adam(learning_rate=taxa_apren)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

tuner = kt.Hyperband(KerasTunnerLSTM1,
                     objective='val_mae',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir2',
                     project_name='salinidade')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(pr_teste, sal_teste,epochs=10,batch_size=120,validation_split=0.2, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 04m 22s]
val_mae: 0.8743407130241394

Best val_mae So Far: 0.8601447939872742
Total elapsed time: 00h 43m 55s
INFO:tensorflow:Oracle triggered exit


###5.1 Resultado

In [ ]:
best_hps.values

{'learning_rate': 0.007,
 'tuner/bracket': 0,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units': 30}

##6. Melhor época para os dados de teste

###6.1 criar função

In [ ]:
def KerasTunnerLSTM():
  input = tf.keras.layers.Input(shape=(2,1), name='Bat_Temp')
  lstm = tf.keras.layers.LSTM(units=30,activation='relu')(input)
  output = tf.keras.layers.Dense(1,name='salinidade')(lstm)
  modelo = tf.keras.Model(inputs=input, outputs=output)
  adan = tf.keras.optimizers.Adam(learning_rate=0.007)
  modelo.compile(optimizer=adan, loss='mse', metrics=['mae','mse'])
  return modelo

###6.2 treinar o modelo com os dados de teste para a melhor epoca

In [ ]:
modelo  = KerasTunnerLSTM()
best_epoch = modelo.fit(pr_treino,sal_treino,epochs=100,batch_size=120)
val_acc_per_epoch = best_epoch.history['mae']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch))+1
modelo.fit(pr_treino,sal_treino,epochs=best_epoch,batch_size=120)
modelo.evaluate(pr_teste,sal_teste)

Epoch 1/100
10093/10093 [==============================] - 37s 4ms/step - loss: 10.9505 - mae: 1.2032 - mse: 10.9505
Epoch 2/100
10093/10093 [==============================] - 36s 4ms/step - loss: 4.5247 - mae: 0.9548 - mse: 4.5247
Epoch 3/100
10093/10093 [==============================] - 36s 4ms/step - loss: 4.4456 - mae: 0.9384 - mse: 4.4456
Epoch 4/100
10093/10093 [==============================] - 35s 3ms/step - loss: 4.4166 - mae: 0.9295 - mse: 4.4166
Epoch 5/100
10093/10093 [==============================] - 35s 3ms/step - loss: 4.3964 - mae: 0.9222 - mse: 4.3964
Epoch 6/100
10093/10093 [==============================] - 36s 4ms/step - loss: 4.3889 - mae: 0.9197 - mse: 4.3889
Epoch 7/100
10093/10093 [==============================] - 36s 4ms/step - loss: 4.3801 - mae: 0.9164 - mse: 4.3801
Epoch 8/100
10093/10093 [==============================] - 35s 4ms/step - loss: 4.3693 - mae: 0.9116 - mse: 4.3693
Epoch 9/100
10093/10093 [==============================] - 35s 3ms/step - loss

[4.306046485900879, 0.8695584535598755, 4.306046485900879]

#RESULTADO
MAE: 0.869

MSE: 4.306

EPOCH: 85

TAXA DE APRENDIZAGEM: 0.007

NEURONIOS: 30